# Distributed Round Robin Group Chat
This notebook demonstrates how to implement a distributed round-robin group chat. In this example we will create three {py:class} `WorkerAgentRuntime` runtime object, two chat participant and a group chat manager, and each of them will connect to an instance of  `WorkerAgentRuntimeHost` which implements the required grpc server.
You would only need to provide `OPENAI_API_KEY` variable below to be able to run this notebook.

In [1]:
import os

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [2]:
import asyncio
from typing import List

from autogen_core.application import WorkerAgentRuntime, WorkerAgentRuntimeHost
from autogen_core.base import MessageContext
from autogen_core.base._serialization import try_get_known_serializers_for_type
from autogen_core.base._topic import TopicId
from autogen_core.components import (
    RoutedAgent,
    message_handler,
    type_subscription,
)
from autogen_core.components._default_subscription import DefaultSubscription
from autogen_core.components._type_subscription import TypeSubscription
from autogen_core.components.models import (
    AssistantMessage,
    ChatCompletionClient,
    LLMMessage,
    OpenAIChatCompletionClient,
    SystemMessage,
    UserMessage,
)
from pydantic import BaseModel


class GroupChatMessage(BaseModel):
    """Implements a sample message sent by an LLM agent"""

    body: LLMMessage


class RequestToSpeak(BaseModel):
    """Message type for agents to speak"""

    pass

# Group Chat Manager
- **group chat manager** implements a round-robing algorithm to ask agents to talk, one at a time. It sends `SpeakRequest` requests to agent topics. This agent subscrives to topic type **agent_result** to receive participants messages.

# Participants
- **data scientist**: Listens to `SpeakRequest` sent by manager and publishes a LLM API call response as `GroupChatMessage`. This agent subscribes to `topic_type="data_scientist", agent_type="data_scientist_ag_type"`
- **art director**:  Listens to `SpeakRequest` sent by manager and publishes a LLM API call response as `GroupChatMessage`. This agent subscribes to `topic_type="art_director", agent_type="art_director_ag_type"`


In [3]:
class ParticipantAgent(RoutedAgent):
    def __init__(self, name: str, description: str, model_client: ChatCompletionClient, system_message: str) -> None:
        super().__init__("A Participant")
        self._name = name
        self._description = description
        self._model_client = model_client
        self._chat_history: List[LLMMessage] = [SystemMessage(system_message)]

    @message_handler
    async def handle_message(self, message: GroupChatMessage, ctx: MessageContext) -> None:
        print(f"received {message}")
        self._chat_history.append(message.body)

    @message_handler
    async def handle_request_to_speak(self, message: RequestToSpeak, ctx: MessageContext) -> None:
        completion = await self._model_client.create(self._chat_history)
        assert isinstance(completion.content, str)
        self._chat_history.append(AssistantMessage(content=completion.content, source=self._name))
        print(f"\n{'-'*80}\n{self._name} ({id(self)}):\n{completion.content}")
        await asyncio.sleep(2)
        await self.publish_message(
            GroupChatMessage(body=UserMessage(content=completion.content, source=self._name)),
            TopicId(type="agent_result", source="participant"),
        )


@type_subscription(topic_type="agent_result")
class GroupChatManager(RoutedAgent):
    def __init__(self, participant_topic_types: List[str], max_rounds: int = 3) -> None:
        super().__init__("Group chat manager")
        self._num_rounds = 1
        self._participant_topic_types = participant_topic_types
        self._chat_history: List[GroupChatMessage] = []
        self._max_rounds = max_rounds

    @message_handler
    async def handle_message(self, message: GroupChatMessage, ctx: MessageContext) -> None:
        self._chat_history.append(message)
        assert isinstance(message.body, UserMessage)
        if self._num_rounds >= self._max_rounds:
            print(f"\n\n\n ---> Finished running {self._num_rounds} rounds! <---")
            await asyncio.sleep(1)
            return
        speaker_topic_type = self._participant_topic_types[self._num_rounds % len(self._participant_topic_types)]
        self._num_rounds += 1
        print(f"\n{'-'*80}\n Manager ({id(self)}): Asking {speaker_topic_type} to speak")
        await self.publish_message(RequestToSpeak(), TopicId(type=speaker_topic_type, source="manager"))

In [4]:
# Setting up gRPC server through `WorkerAgentRuntimeHost`
host_address = "localhost:50060"
host = WorkerAgentRuntimeHost(address=host_address)
serializers = [
    try_get_known_serializers_for_type(GroupChatMessage),
    try_get_known_serializers_for_type(RequestToSpeak),
]

host.start()
print(f"Distributed Host is now running and listening for connection at {host_address}")

# Add data scientist runtime
data_scientist_runtime = WorkerAgentRuntime(host_address=host_address)
data_scientist_runtime.add_message_serializer(serializers)  # type: ignore[arg-type]
data_scientist_runtime.start()
await data_scientist_runtime.add_subscription(
    TypeSubscription(topic_type="data_scientist", agent_type="data_scientist_ag_type")
)
await ParticipantAgent.register(
    data_scientist_runtime,
    "data_scientist_ag_type",
    lambda: ParticipantAgent(
        name="data_scientist_agent",
        description="A data scientist in a discussion",
        system_message="You are a data scientist in a conversation, provide 1-2 sentences in a discussion",
        model_client=OpenAIChatCompletionClient(model="gpt-4o-mini", api_key=OPENAI_API_KEY),
    ),
)

# Add art director runtime
art_director_runtime = WorkerAgentRuntime(host_address=host_address)
art_director_runtime.add_message_serializer(serializers)  # type: ignore[arg-type]
art_director_runtime.start()
await art_director_runtime.add_subscription(
    TypeSubscription(topic_type="art_director", agent_type="art_director_ag_type")
)

await ParticipantAgent.register(
    art_director_runtime,
    "art_director_ag_type",
    lambda: ParticipantAgent(
        name="art_director_agent",
        description="A art director in a discussion",
        system_message="You are a art director in a conversation, provide 1-2 sentences in a discussion",
        model_client=OpenAIChatCompletionClient(model="gpt-4o-mini", api_key=OPENAI_API_KEY),
    ),
)

# Add group chat manager runtime
group_chat_manager_runtime = WorkerAgentRuntime(host_address=host_address)
group_chat_manager_runtime.add_message_serializer(serializers)  # type: ignore[arg-type]
group_chat_manager_runtime.start()

await GroupChatManager.register(
    group_chat_manager_runtime,
    "group_chat_manager_ag_type",
    lambda: GroupChatManager(participant_topic_types=["data_scientist", "art_director"], max_rounds=3),
)


# Send a message to data scientist to get started
await group_chat_manager_runtime.publish_message(RequestToSpeak(), TopicId(type="data_scientist", source="manager"))

await asyncio.sleep(10)  # Wait for messages to propagate
await host.stop()

Distributed Host is now running and listening for connection at localhost:50060

--------------------------------------------------------------------------------
data_scientist_agent (4642320400):
As a data scientist, I can help analyze trends and patterns in your data to drive informed decisions. If you have specific datasets or questions in mind, feel free to share, and we can dive deeper into the analysis together!

--------------------------------------------------------------------------------
 Manager (4642640912): Asking art_director to speak

--------------------------------------------------------------------------------
art_director_agent (4642837840):
As an art director, I believe that our visual narrative should reflect the core message of the project while also engaging the audience on an emotional level. It's crucial that each design element we choose enhances the overall aesthetic and storytelling we aim to convey.

-------------------------------------------------------